In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 36674792
paper_name = 'piconpages_munoz_2023' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [15]:
original_data1 = pd.read_csv('raw_data/tableS1.txt', header=None, names=['orfs','genes'], sep=' ')
original_data2 = pd.read_csv('raw_data/tableS2.txt', header=None, names=['orfs','genes'], sep=' ')

In [16]:
print('Original data dimensions: %d x %d' % (original_data1.shape))
print('Original data dimensions: %d x %d' % (original_data2.shape))

Original data dimensions: 141 x 2
Original data dimensions: 312 x 2


In [17]:
original_data1['orf'] = original_data1['orfs'].astype(str)
original_data2['orf'] = original_data2['orfs'].astype(str)

In [18]:
# Eliminate all white spaces & capitalize
original_data1['orf'] = clean_orf(original_data1['orf'])
original_data2['orf'] = clean_orf(original_data2['orf'])

In [19]:
# Translate to ORFs 
original_data1['orf'] = translate_sc(original_data1['orf'], to='orf')
original_data2['orf'] = translate_sc(original_data2['orf'], to='orf')

In [20]:
# Make sure everything translated ok
t = looks_like_orf(original_data1['orf'])
print(original_data1.loc[~t,])

Empty DataFrame
Columns: [orfs, genes, orf]
Index: []


In [21]:
# Make sure everything translated ok
t = looks_like_orf(original_data2['orf'])
print(original_data2.loc[~t,])

Empty DataFrame
Columns: [orfs, genes, orf]
Index: []


In [22]:
original_data1['data'] = 1
original_data2['data'] = -1

In [23]:
original_data1.set_index('orf', inplace=True)
original_data2.set_index('orf', inplace=True)

In [32]:
original_data = pd.concat([original_data1[['data']], original_data2[['data']]], axis=0)

In [33]:
original_data.shape

(453, 1)

In [34]:
original_data.head()

,data
orf,
YAL011W,1
YAL021C,1
YAL040C,1
YAL044C,1
YAL060W,1


In [35]:
original_data = original_data.groupby(original_data.index).mean()

In [36]:
original_data.shape

(453, 1)

# Prepare the final dataset

In [37]:
data = original_data.copy()

In [38]:
dataset_ids = [22240]
datasets = datasets.reindex(index=dataset_ids)

In [39]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [40]:
data.head()

dataset_id,22240
data_type,value
orf,
YAL011W,1
YAL021C,1
YAL039C,-1
YAL040C,1
YAL044C,1


## Subset to the genes currently in SGD

In [41]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 1


In [42]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,22240
,data_type,value
gene_id,orf,
9,YAL011W,1
19,YAL021C,1
37,YAL039C,-1
38,YAL040C,1
42,YAL044C,1


# Normalize

In [43]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [44]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [45]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id      22240          
data_type       value    valuez
gene_id orf                    
9       YAL011W     1  3.225604
19      YAL021C     1  3.225604
37      YAL039C    -1 -3.161092
38      YAL040C     1  3.225604
42      YAL044C     1  3.225604

# Print out

In [46]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')